# Import and Read data


In [1]:
import sys
sys.path.append('.')  # Add current directory to path
from build_graph_and_train import *

/opt/conda/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
partition = 100

In [3]:
df = pd.read_csv(f"../../../../data/top30groups/DNDF_OneHotLongLatCombined/combined/combined{partition}.csv")

In [4]:
import os 
if not os.path.isdir(f"Results{partition}"):
    os.mkdir(f"Results{partition}")

# Create longlat feature

In [5]:
geodata = ['longitude', 'latitude']
combined_geo = df.copy()
combined_geo['longlat'] = list(zip(df['longitude'], df['latitude']))
combined_geo = combined_geo.drop(columns=geodata)

In [6]:
import ast

def to_tuple_if_needed(val):
    if isinstance(val, str):
        return ast.literal_eval(val)
    return val  # already a tuple

combined_geo['longlat'] = combined_geo['longlat'].apply(to_tuple_if_needed)

In [7]:
combined_geo.dtypes

extended         int64
vicinity         int64
multiple       float64
success          int64
suicide          int64
attacktype1      int64
targtype1        int64
target1          int64
individual       int64
weaptype1        int64
nkill          float64
property         int64
ishostkid      float64
gname           object
longlat         object
dtype: object

# Weapon type prediction

In [8]:
label_index = {g: i for i, g in enumerate(sorted(df['gname'].unique()))}
continuous_col = 'weaptype1'
data, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, row_to_node_index = build_graph_data(combined_geo, label_index, continuous_col=continuous_col)
args = {
    'partition': f"gtd{partition}",
    'embed_dim': 16,
    'lr': 0.01,
    'epochs': 1500,
    'feat_dropout': 0,
    'n_tree': 80,
    'tree_depth': 10,
    'tree_feature_rate': 0.5,
    'n_class': len(label_index)  
}
best_acc, best_epoch = train_joint(data, data.edge_index, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, args, row_to_node_index)

with open(f"Results{partition}/Results_{continuous_col}_prediction", "w") as f:
    f.write(f"Best acc: {best_acc} in epoch {best_epoch} for {continuous_col} prediction")

/home/jovyan/buggy_GTD/GTD_2025/Codes/Baselines/graphrfi/GraphRfi_OneHotLongLat/build_graph_and_train.py:119: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_nrf = torch.tensor(y_nrf, dtype=torch.long).to(device)


Epoch 01 | GCN MSE Loss: 45.5031 | NRF Loss: 3.4013 | JOINT Loss: 48.9044 | NRF Acc: 0.0343
Epoch 02 | GCN MSE Loss: 29.3087 | NRF Loss: 3.4002 | JOINT Loss: 32.7089 | NRF Acc: 0.0560
Epoch 03 | GCN MSE Loss: 28.3092 | NRF Loss: 3.3894 | JOINT Loss: 31.6987 | NRF Acc: 0.0412
Epoch 04 | GCN MSE Loss: 24.4909 | NRF Loss: 3.3920 | JOINT Loss: 27.8829 | NRF Acc: 0.0698
Epoch 05 | GCN MSE Loss: 20.6638 | NRF Loss: 3.3910 | JOINT Loss: 24.0548 | NRF Acc: 0.0766
Epoch 06 | GCN MSE Loss: 19.2384 | NRF Loss: 3.3859 | JOINT Loss: 22.6243 | NRF Acc: 0.0772
Epoch 07 | GCN MSE Loss: 18.2635 | NRF Loss: 3.3821 | JOINT Loss: 21.6456 | NRF Acc: 0.0692
Epoch 08 | GCN MSE Loss: 15.6281 | NRF Loss: 3.3810 | JOINT Loss: 19.0091 | NRF Acc: 0.0800
Epoch 09 | GCN MSE Loss: 12.6477 | NRF Loss: 3.3783 | JOINT Loss: 16.0260 | NRF Acc: 0.0823
Epoch 10 | GCN MSE Loss: 11.2480 | NRF Loss: 3.3757 | JOINT Loss: 14.6238 | NRF Acc: 0.0806
Epoch 11 | GCN MSE Loss: 11.5035 | NRF Loss: 3.3728 | JOINT Loss: 14.8763 | NRF 

# Fatality rate


In [9]:
label_index = {g: i for i, g in enumerate(sorted(df['gname'].unique()))}
continuous_col = 'nkill'
data, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, row_to_node_index = build_graph_data(combined_geo, label_index, continuous_col=continuous_col)
args = {
    'partition': f"gtd{partition}",
    'embed_dim': 16,
    'lr': 0.01,
    'epochs': 1500,
    'feat_dropout': 0,
    'n_tree': 80,
    'tree_depth': 10,
    'tree_feature_rate': 0.5,
    'n_class': len(label_index)  
}
best_acc, best_epoch = train_joint(data, data.edge_index, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, args, row_to_node_index)

with open(f"Results{partition}/Results_{continuous_col}_prediction", "w") as f:
    f.write(f"Best acc: {best_acc} in epoch {best_epoch} for {continuous_col} prediction")

/home/jovyan/buggy_GTD/GTD_2025/Codes/Baselines/graphrfi/GraphRfi_OneHotLongLat/build_graph_and_train.py:119: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_nrf = torch.tensor(y_nrf, dtype=torch.long).to(device)


Epoch 01 | GCN MSE Loss: 175.9436 | NRF Loss: 3.4010 | JOINT Loss: 179.3447 | NRF Acc: 0.0532
Epoch 02 | GCN MSE Loss: 147.9390 | NRF Loss: 3.3912 | JOINT Loss: 151.3302 | NRF Acc: 0.0549
Epoch 03 | GCN MSE Loss: 131.4868 | NRF Loss: 3.3879 | JOINT Loss: 134.8748 | NRF Acc: 0.0212
Epoch 04 | GCN MSE Loss: 120.6968 | NRF Loss: 3.3874 | JOINT Loss: 124.0843 | NRF Acc: 0.0206
Epoch 05 | GCN MSE Loss: 114.1186 | NRF Loss: 3.3907 | JOINT Loss: 117.5094 | NRF Acc: 0.0257
Epoch 06 | GCN MSE Loss: 111.4053 | NRF Loss: 3.3902 | JOINT Loss: 114.7955 | NRF Acc: 0.0343
Epoch 07 | GCN MSE Loss: 111.4976 | NRF Loss: 3.3873 | JOINT Loss: 114.8849 | NRF Acc: 0.0314
Epoch 08 | GCN MSE Loss: 112.5603 | NRF Loss: 3.3857 | JOINT Loss: 115.9460 | NRF Acc: 0.0263
Epoch 09 | GCN MSE Loss: 112.8960 | NRF Loss: 3.3847 | JOINT Loss: 116.2807 | NRF Acc: 0.0269
Epoch 10 | GCN MSE Loss: 111.9186 | NRF Loss: 3.3829 | JOINT Loss: 115.3015 | NRF Acc: 0.0240
Epoch 11 | GCN MSE Loss: 110.0619 | NRF Loss: 3.3792 | JOINT

# Target Type

In [10]:
label_index = {g: i for i, g in enumerate(sorted(df['gname'].unique()))}
continuous_col = 'targtype1'
data, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, row_to_node_index = build_graph_data(combined_geo, label_index, continuous_col=continuous_col)
args = {
    'partition': f"gtd{partition}",
    'embed_dim': 16,
    'lr': 0.01,
    'epochs': 1500,
    'feat_dropout': 0,
    'n_tree': 80,
    'tree_depth': 10,
    'tree_feature_rate': 0.5,
    'n_class': len(label_index)  
}
best_acc, best_epoch = train_joint(data, data.edge_index, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, args, row_to_node_index)

with open(f"Results{partition}/Results_{continuous_col}_prediction", "w") as f:
    f.write(f"Best acc: {best_acc} in epoch {best_epoch} for {continuous_col} prediction")

/home/jovyan/buggy_GTD/GTD_2025/Codes/Baselines/graphrfi/GraphRfi_OneHotLongLat/build_graph_and_train.py:119: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_nrf = torch.tensor(y_nrf, dtype=torch.long).to(device)


Epoch 01 | GCN MSE Loss: 1581.0328 | NRF Loss: 3.4017 | JOINT Loss: 1584.4346 | NRF Acc: 0.0417
Epoch 02 | GCN MSE Loss: 1325.6042 | NRF Loss: 3.3977 | JOINT Loss: 1329.0020 | NRF Acc: 0.0417
Epoch 03 | GCN MSE Loss: 1100.0360 | NRF Loss: 3.3968 | JOINT Loss: 1103.4327 | NRF Acc: 0.0475
Epoch 04 | GCN MSE Loss: 901.8552 | NRF Loss: 3.3960 | JOINT Loss: 905.2512 | NRF Acc: 0.0475
Epoch 05 | GCN MSE Loss: 730.2801 | NRF Loss: 3.3941 | JOINT Loss: 733.6742 | NRF Acc: 0.0457
Epoch 06 | GCN MSE Loss: 583.0858 | NRF Loss: 3.3908 | JOINT Loss: 586.4766 | NRF Acc: 0.0463
Epoch 07 | GCN MSE Loss: 458.0722 | NRF Loss: 3.3890 | JOINT Loss: 461.4611 | NRF Acc: 0.0463
Epoch 08 | GCN MSE Loss: 353.1914 | NRF Loss: 3.3852 | JOINT Loss: 356.5766 | NRF Acc: 0.0469
Epoch 09 | GCN MSE Loss: 267.6630 | NRF Loss: 3.3863 | JOINT Loss: 271.0493 | NRF Acc: 0.0463
Epoch 10 | GCN MSE Loss: 199.9705 | NRF Loss: 3.3872 | JOINT Loss: 203.3577 | NRF Acc: 0.0469
Epoch 11 | GCN MSE Loss: 148.5455 | NRF Loss: 3.3890 |

# Attack type

In [11]:
label_index = {g: i for i, g in enumerate(sorted(df['gname'].unique()))}
continuous_col = 'attacktype1'
data, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, row_to_node_index = build_graph_data(combined_geo, label_index, continuous_col=continuous_col)
args = {
    'partition': f"gtd{partition}",
    'embed_dim': 16,
    'lr': 0.01,
    'epochs': 1500,
    'feat_dropout': 0,
    'n_tree': 80,
    'tree_depth': 10,
    'tree_feature_rate': 0.5,
    'n_class': len(label_index)  
}
best_acc, best_epoch = train_joint(data, data.edge_index, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, args, row_to_node_index)

with open(f"Results{partition}/Results_{continuous_col}_prediction", "w") as f:
    f.write(f"Best acc: {best_acc} in epoch {best_epoch} for {continuous_col} prediction")

/home/jovyan/buggy_GTD/GTD_2025/Codes/Baselines/graphrfi/GraphRfi_OneHotLongLat/build_graph_and_train.py:119: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_nrf = torch.tensor(y_nrf, dtype=torch.long).to(device)


Epoch 01 | GCN MSE Loss: 229.5023 | NRF Loss: 3.4007 | JOINT Loss: 232.9030 | NRF Acc: 0.0560
Epoch 02 | GCN MSE Loss: 129.6381 | NRF Loss: 3.3941 | JOINT Loss: 133.0322 | NRF Acc: 0.0555
Epoch 03 | GCN MSE Loss: 64.5118 | NRF Loss: 3.3904 | JOINT Loss: 67.9022 | NRF Acc: 0.0360
Epoch 04 | GCN MSE Loss: 31.1436 | NRF Loss: 3.3956 | JOINT Loss: 34.5392 | NRF Acc: 0.0292
Epoch 05 | GCN MSE Loss: 23.2358 | NRF Loss: 3.3986 | JOINT Loss: 26.6344 | NRF Acc: 0.0503
Epoch 06 | GCN MSE Loss: 30.9380 | NRF Loss: 3.3965 | JOINT Loss: 34.3345 | NRF Acc: 0.0738
Epoch 07 | GCN MSE Loss: 42.7834 | NRF Loss: 3.3914 | JOINT Loss: 46.1748 | NRF Acc: 0.0806
Epoch 08 | GCN MSE Loss: 50.2873 | NRF Loss: 3.3883 | JOINT Loss: 53.6755 | NRF Acc: 0.0806
Epoch 09 | GCN MSE Loss: 50.4171 | NRF Loss: 3.3845 | JOINT Loss: 53.8016 | NRF Acc: 0.0812
Epoch 10 | GCN MSE Loss: 44.2770 | NRF Loss: 3.3814 | JOINT Loss: 47.6583 | NRF Acc: 0.0783
Epoch 11 | GCN MSE Loss: 34.6736 | NRF Loss: 3.3799 | JOINT Loss: 38.0536 | 